In [1]:
import pandas as pd
import numpy as np
import json
import gzip
import os
import os.path
from os import listdir
import multiprocessing
from tqdm import tqdm
import mmap
from multiprocessing import Pool
import seaborn as sns
import matplotlib.pyplot as plt
import py_entitymatching as em
import py_stringmatching as sm
import scipy
from scipy.io import arff
from sklearn import preprocessing
#from sklearn.model_selection import train_test_split
from skmultilearn.model_selection import iterative_train_test_split
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

In [2]:
#read table to dataframe and create train/test split with equal distribution of labels within table
all_tables = pd.read_csv('../../src/data/data/CSV_files/relevant_tables_more_than_20.csv')
all_tables = all_tables.drop(columns={'Unnamed: 0'})
#pd.set_option('display.max_columns', None)
all_tables

,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,...,Restaurant_menu,Restaurant_name,Restaurant_openinghours,Restaurant_openinghoursspecification,Restaurant_pricerange,Restaurant_servescuisine,Restaurant_telephone,TVEpisode_episodenumber,TVEpisode_name,TVEpisode_partofseries
0,Book_12min.com_September2020.json.gz,Book,366.0,9.0,5.0,166,0.286885,0.090710,830.0,1,...,0,0,0,0,0,0,0,0,0,0
1,Book_19shuwu.com_September2020.json.gz,Book,250.0,8.0,5.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0,0,0
2,Book_1carpetcleaning.co.uk_September2020.json.gz,Book,1004.0,12.0,9.0,0,0.000000,0.000000,0.0,1,...,0,0,0,0,0,0,0,0,0,0
3,Book_2014brazil.co.uk_September2020.json.gz,Book,387.0,11.0,9.0,0,0.000000,0.000000,0.0,1,...,0,0,0,0,0,0,0,0,0,0
4,Book_24symbols.com_September2020.json.gz,Book,5171.0,8.0,5.0,17,0.001740,0.000658,85.0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54185,TVEpisode_yara-online.org_September2020.json.gz,TVEpisode,40.0,9.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0,1,1
54186,TVEpisode_yesmoviess.com_September2020.json.gz,TVEpisode,37.0,9.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0,1,1
54187,TVEpisode_zazangels.com_September2020.json.gz,TVEpisode,38.0,8.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0,1,1
54188,TVEpisode_zebrahead.org_September2020.json.gz,TVEpisode,80.0,7.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0,1,1


In [3]:
all_tables.loc[all_tables['num_rows'] <= 40, 'Small'] = 1 
all_tables.loc[(all_tables['num_rows'] > 40) & (all_tables['num_rows'] <= 160), 'Medium'] = 1
all_tables.loc[all_tables['num_rows'] > 160, 'Large'] = 1 
all_tables = all_tables.fillna(0)

In [4]:
 liste  = list(all_tables.iloc[:,1:2])
 liste2 = list(all_tables.iloc[:,9:])
 liste = liste + liste2
 liste

['Class',
 'Book_aggregaterating',
 'Book_author',
 'Book_bookedition',
 'Book_bookformat',
 'Book_datepublished',
 'Book_genre',
 'Book_inlanguage',
 'Book_isbn',
 'Book_name',
 'Book_numberofpages',
 'Book_offers',
 'Book_publisher',
 'CreativeWork_aggregaterating',
 'CreativeWork_alternativeheadline',
 'CreativeWork_articlebody',
 'CreativeWork_author',
 'CreativeWork_commentcount',
 'CreativeWork_copyrightholder',
 'CreativeWork_copyrightyear',
 'CreativeWork_creator',
 'CreativeWork_datecreated',
 'CreativeWork_datemodified',
 'CreativeWork_datepublished',
 'CreativeWork_genre',
 'CreativeWork_headline',
 'CreativeWork_inlanguage',
 'CreativeWork_interactioncount',
 'CreativeWork_interactionstatistic',
 'CreativeWork_interactiontype',
 'CreativeWork_keywords',
 'CreativeWork_mainentityofpage',
 'CreativeWork_name',
 'CreativeWork_offers',
 'CreativeWork_publisher',
 'CreativeWork_version',
 'Event_doortime',
 'Event_duration',
 'Event_enddate',
 'Event_eventattendancemode',
 'Even

In [5]:
all_tables = all_tables.fillna(0)

In [5]:
path = '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/data/CSV_files'
tables = pd.read_csv(path + '/all_files_cleaned.csv')
tables = tables.drop(columns={'Unnamed: 0'})
columns_sorted = tables.groupby('filename')['column_name'].apply(list).reset_index(name='listofcolumns')
columns_sorted.filename = columns_sorted.filename.str.split('_', expand = True)
columns_sorted.index = columns_sorted['filename']
columns_sorted

,filename,listofcolumns
filename,,
Book,Book,"[name, author, isbn, publisher, offers, datepublished, numberofpages, bookformat, inlanguage, ge..."
CreativeWork,CreativeWork,"[headline, author, datepublished, datemodified, name, publisher, mainentityofpage, interactionst..."
Event,Event,"[name, enddate, location, performer, organizer, eventstatus, eventattendancemode, duration, door..."
Hotel,Hotel,"[name, address, telephone, pricerange, geo, aggregaterating, starrating, email]"
LocalBusiness,LocalBusiness,"[name, address, telephone, geo, pricerange, aggregaterating, email, sameas, faxnumber, openingho..."
MusicAlbum,MusicAlbum,"[name, byartist, track, numtracks]"
MusicRecording,MusicRecording,"[name, duration, byartist, inalbum, datepublished, genre, datecreated]"
Person,Person,"[mainentityofpage, jobtitle, birthdate, givenname, familyname, telephone, email, deathdate, addi..."
Place,Place,"[name, geo, telephone, sameas, openinghoursspecification]"


In [7]:
#Split all Tables in large train and test set
#def traintestsplit():
#for index in columns_sorted.filename:
#    index_df = pd.DataFrame()
#    liste = []
#    index_df = small[small['Class']==index]
    #display(index_df)
#    liste  = list([col for col in small if col.startswith(index+'_')])
#liste = list([col for col in small if col.startswith(index+'_')])
X = np.array(all_tables['filename'])
y = np.array(all_tables[liste])
msss = MultilabelStratifiedShuffleSplit(n_splits=3, test_size=0.18, random_state=42)
for train_index, test_index in msss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#print(test_index)
train_set = set(X_train)
print(train_set)
test_set = set(X_test)
print(test_set)
for index in columns_sorted.filename:
    inputdir = '../../src/data/schemafiltereddata/'+index+'/'
    outputdir = '../../src/data/schemafiltereddata/TrainTestTables/Large/'
    files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]  
    for file in files:
        if file in train_set:
            #for file in train_set:
            #print(file)
            df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
            df.to_json(os.path.join(outputdir+'Train/'+file), compression='gzip', orient='records', lines=True)
        print(file)
        if file in test_set:
        #for file in test_set:
            #print(file)
            df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
            df.to_json(os.path.join(outputdir+'Test/'+file), compression='gzip', orient='records', lines=True)
        print(file)
print(len([file for file in os.listdir(outputdir+'Test/') if file.endswith('.json.gz')]))

TRAIN: [    0     1     3 ... 54187 54188 54189] TEST: [    2    15    18 ... 54177 54184 54186]
TRAIN: [    0     2     3 ... 54184 54187 54189] TEST: [    1     6    13 ... 54185 54186 54188]
TRAIN: [    0     2     3 ... 54186 54187 54189] TEST: [    1     7     9 ... 54176 54182 54188]
{'Person_advantaira.com_September2020.json.gz', 'Recipe_veggietracker.com_September2020.json.gz', 'MusicRecording_brittawolfert.com_September2020.json.gz', 'Person_bramille.com_September2020.json.gz', 'Person_cliffordchance.com_September2020.json.gz', 'Product_paperose.com.my_September2020.json.gz', 'Person_foundsfuneralhome.com_September2020.json.gz', 'Product_honeyhairco.com_September2020.json.gz', 'Product_getbeast.com_September2020.json.gz', 'Hotel_hotel-rv.com_September2020.json.gz', 'Product_farahasmar.com_September2020.json.gz', 'Product_incrediwearcanada.com_September2020.json.gz', 'Event_wellnessloungesd.com_September2020.json.gz', 'CreativeWork_ankiros.com_September2020.json.gz', 'LocalBusi

In [6]:
listeTe = os.listdir(path='../../src/data/schemafiltereddata/TrainTestTables/Large/Test/')
listeTr = os.listdir(path='../../src/data/schemafiltereddata/TrainTestTables/Large/Train/')

In [14]:
#len(listeTe)
len(listeTr)

44435

In [7]:
large_train = all_tables[all_tables['filename'].isin(listeTr)]
large_train.to_csv('../../src/data/schemafiltereddata/TrainTestTables/Large/Training_Files.csv')
large_train


,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,...,Restaurant_openinghoursspecification,Restaurant_pricerange,Restaurant_servescuisine,Restaurant_telephone,TVEpisode_episodenumber,TVEpisode_name,TVEpisode_partofseries,Small,Medium,Large
0,Book_12min.com_September2020.json.gz,Book,366.0,9.0,5.0,166,0.286885,0.090710,830.0,1,...,0,0,0,0,0,0,0,0.0,0.0,1.0
2,Book_1carpetcleaning.co.uk_September2020.json.gz,Book,1004.0,12.0,9.0,0,0.000000,0.000000,0.0,1,...,0,0,0,0,0,0,0,0.0,0.0,1.0
3,Book_2014brazil.co.uk_September2020.json.gz,Book,387.0,11.0,9.0,0,0.000000,0.000000,0.0,1,...,0,0,0,0,0,0,0,0.0,0.0,1.0
4,Book_24symbols.com_September2020.json.gz,Book,5171.0,8.0,5.0,17,0.001740,0.000658,85.0,0,...,0,0,0,0,0,0,0,0.0,0.0,1.0
5,Book_2d-market.com_September2020.json.gz,Book,262.0,19.0,7.0,76,0.129771,0.041439,532.0,0,...,0,0,0,0,0,0,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54184,TVEpisode_yahooboard.net_September2020.json.gz,TVEpisode,55.0,9.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,1,1,0.0,1.0,0.0
54185,TVEpisode_yara-online.org_September2020.json.gz,TVEpisode,40.0,9.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,1,1,1.0,0.0,0.0
54186,TVEpisode_yesmoviess.com_September2020.json.gz,TVEpisode,37.0,9.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,1,1,1.0,0.0,0.0
54187,TVEpisode_zazangels.com_September2020.json.gz,TVEpisode,38.0,8.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,1,1,1.0,0.0,0.0


In [8]:
large_test = all_tables[all_tables['filename'].isin(listeTe)]
large_test.to_csv('../../src/data/schemafiltereddata/TrainTestTables/Large/Test_Files.csv')
large_test

,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,...,Restaurant_openinghoursspecification,Restaurant_pricerange,Restaurant_servescuisine,Restaurant_telephone,TVEpisode_episodenumber,TVEpisode_name,TVEpisode_partofseries,Small,Medium,Large
1,Book_19shuwu.com_September2020.json.gz,Book,250.0,8.0,5.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,0,0,0.0,0.0,1.0
7,Book_4thestate.co.uk_September2020.json.gz,Book,1336.0,10.0,6.0,523,0.391467,0.065245,3138.0,0,...,0,0,0,0,0,0,0,0.0,0.0,1.0
9,Book_80mundos.com_September2020.json.gz,Book,2748.0,10.0,7.0,696,0.129185,0.036182,4872.0,0,...,0,0,0,0,0,0,0,0.0,0.0,1.0
21,Book_alexanderstreet.com_September2020.json.gz,Book,197.0,11.0,5.0,146,0.446701,0.148223,730.0,0,...,0,0,0,0,0,0,0,0.0,0.0,1.0
28,Book_apress.com_September2020.json.gz,Book,399.0,8.0,5.0,124,0.308271,0.062155,620.0,0,...,0,0,0,0,0,0,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54165,TVEpisode_univision.com_September2020.json.gz,TVEpisode,3150.0,5.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,1,1,0,0.0,0.0,1.0
54174,TVEpisode_votenova.org_September2020.json.gz,TVEpisode,55.0,9.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,1,1,0.0,1.0,0.0
54176,TVEpisode_watermarkthefilm.com_September2020.json.gz,TVEpisode,40.0,9.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,1,1,1.0,0.0,0.0
54182,TVEpisode_ww2d.org_September2020.json.gz,TVEpisode,54.0,8.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,1,1,0.0,1.0,0.0


In [25]:
#Split large test set into medium test set
#def traintestsplit():

X = np.array(large_test['filename'])
y = np.array(large_test[liste])
msss = MultilabelStratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=42)
for train_index, test_index in msss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#print(test_index)
train_set = set(X_train)
print(train_set)
test_set = set(X_test)
print(test_set)
for index in columns_sorted.filename:
    inputdir = '../../src/data/schemafiltereddata/TrainTestTables/Large/Test/'
    outputdir = '../../src/data/schemafiltereddata/TrainTestTables/Medium/'
    files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]  
    for file in files:
        #if file in train_set:
            #for file in train_set:
            #print(file)
        #    df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
        #    df.to_json(os.path.join(outputdir+'Train/'+file), compression='gzip', orient='records', lines=True)
        #print(file)
        if file in test_set:
        #for file in test_set:
            #print(file)
            df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
            df.to_json(os.path.join(outputdir+'Test/'+file), compression='gzip', orient='records', lines=True)
        print(file)
print(len([file for file in os.listdir(outputdir+'Test/') if file.endswith('.json.gz')]))

In [7]:
medium_listeTe = os.listdir(path='../../src/data/schemafiltereddata/TrainTestTables/Medium/Test/')
medium_listeTe

['Event_davidlohmueller.com_September2020.json.gz',
 'Product_angelmania.net_September2020.json.gz',
 'Product_jd-sports.com.au_September2020.json.gz',
 'Recipe_ozlemsturkishtable.com_September2020.json.gz',
 'Product_fantastichumanhair.com_September2020.json.gz',
 'Recipe_mexicanplease.com_September2020.json.gz',
 'Product_swimsale.com_September2020.json.gz',
 'MusicRecording_shanescheib.com_September2020.json.gz',
 'Product_gusti-leather.co.uk_September2020.json.gz',
 'Restaurant_latelier28.com_September2020.json.gz',
 'Product_ladybaileycaravans.co.uk_September2020.json.gz',
 'Product_ekkip.com_September2020.json.gz',
 'Event_rootszerowastemarket.com_September2020.json.gz',
 'Product_edwards-jewelers.com_September2020.json.gz',
 'Event_pcflv.org_September2020.json.gz',
 'MusicAlbum_lizasongs.com_September2020.json.gz',
 'MusicRecording_sarahgosleereed.com_September2020.json.gz',
 'Recipe_momeatspaleo.com_September2020.json.gz',
 'Product_bokarocake.com_September2020.json.gz',
 'Prod

In [8]:
medium_test = all_tables[all_tables['filename'].isin(medium_listeTe)]
medium_test.to_csv('../../src/data/schemafiltereddata/TrainTestTables/Medium/MTest_Files.csv')
medium_test

,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,...,Restaurant_openinghoursspecification,Restaurant_pricerange,Restaurant_servescuisine,Restaurant_telephone,TVEpisode_episodenumber,TVEpisode_name,TVEpisode_partofseries,Small,Medium,Large
38,Book_atlaskitap.com_September2020.json.gz,Book,2015.0,8.0,5.0,1,0.000496,0.000099,5.0,0,...,0,0,0,0,0,0,0,0.0,0.0,1.0
41,Book_auth-e83.co.uk_September2020.json.gz,Book,947.0,12.0,9.0,0,0.000000,0.000000,0.0,1,...,0,0,0,0,0,0,0,0.0,0.0,1.0
61,Book_berghahnbooks.com_September2020.json.gz,Book,697.0,12.0,9.0,6,0.002869,0.000956,54.0,0,...,0,0,0,0,0,0,0,0.0,0.0,1.0
79,Book_boldtypebooks.com_September2020.json.gz,Book,105.0,12.0,7.0,7,0.057143,0.009524,49.0,0,...,0,0,0,0,0,0,0,0.0,1.0,0.0
106,Book_calamo.com_September2020.json.gz,Book,15746.0,10.0,7.0,6174,0.193891,0.056014,43218.0,0,...,0,0,0,0,0,0,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54036,TVEpisode_ezfilm.net_September2020.json.gz,TVEpisode,101.0,5.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,1,1,0,0.0,1.0,0.0
54055,TVEpisode_grnrngr.com_September2020.json.gz,TVEpisode,910.0,7.0,3.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,1,1,1,0.0,0.0,1.0
54118,TVEpisode_semhade.com_September2020.json.gz,TVEpisode,89.0,5.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,1,1,0,0.0,1.0,0.0
54123,TVEpisode_sinefy.net_September2020.json.gz,TVEpisode,71.0,5.0,2.0,13,0.183099,0.091549,26.0,0,...,0,0,0,0,1,1,0,0.0,1.0,0.0


In [9]:
#Split medium test set into small test set
#def traintestsplit():

X = np.array(medium_test['filename'])
y = np.array(medium_test[liste])
msss = MultilabelStratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=42)
for train_index, test_index in msss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#print(test_index)
train_set = set(X_train)
print(train_set)
test_set = set(X_test)
print(test_set)
for index in columns_sorted.filename:
    inputdir = '../../src/data/schemafiltereddata/TrainTestTables/Medium/Test/'
    outputdir = '../../src/data/schemafiltereddata/TrainTestTables/Small/'
    files = [file for file in os.listdir(inputdir) if file.endswith('.json.gz')]  
    for file in files:
        #if file in train_set:
            #for file in train_set:
            #print(file)
        #    df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
        #    df.to_json(os.path.join(outputdir+'Train/'+file), compression='gzip', orient='records', lines=True)
        #print(file)
        if file in test_set:
        #for file in test_set:
            #print(file)
            df = pd.read_json(os.path.join(inputdir, '{}'.format(file)), compression='gzip', lines=True)
            df.to_json(os.path.join(outputdir+'Test/'+file), compression='gzip', orient='records', lines=True)
        print(file)
print(len([file for file in os.listdir(outputdir+'Test/') if file.endswith('.json.gz')]))

TRAIN: [   0    1    2 ... 1956 1957 1958] TEST: [   9   13   16   17   19   26   28   33   35   38   46   51   53   55
   62   64   79   89   91  103  108  114  116  117  122  130  133  134
  143  145  149  150  151  153  154  157  161  165  190  193  200  201
  202  214  215  216  217  223  225  227  229  230  232  238  252  257
  263  276  281  293  295  299  302  315  317  320  328  337  345  347
  349  376  378  384  385  386  389  392  395  399  400  401  403  409
  419  430  431  441  443  449  454  466  468  470  472  473  484  485
  488  492  498  501  508  509  512  517  519  520  528  531  537  539
  540  541  546  556  563  564  565  574  586  616  625  627  635  640
  641  642  646  647  648  653  657  659  673  676  683  687  690  696
  699  703  717  728  735  738  739  740  742  750  751  758  761  763
  766  768  773  775  776  779  791  795  797  800  801  804  805  806
  837  850  852  853  854  871  873  875  877  880  891  895  899  913
  919  920  929  930  934  9

In [4]:
small_listeTe = os.listdir(path='../../src/data/schemafiltereddata/TrainTestTables/Small/Test/')
small_listeTe

['Product_ladybaileycaravans.co.uk_September2020.json.gz',
 'MusicAlbum_lizasongs.com_September2020.json.gz',
 'Place_thetravellingfool.com_September2020.json.gz',
 'Product_sportforyou.net_September2020.json.gz',
 'Person_homeswithlopez.com_September2020.json.gz',
 'Product_keratinbeauty.com_September2020.json.gz',
 'Event_wackenbus.com_September2020.json.gz',
 'Recipe_casalagotastingroom.com_September2020.json.gz',
 'Product_zebraz.com_September2020.json.gz',
 'MusicRecording_alcardillo.com_September2020.json.gz',
 'CreativeWork_fractracker.org_September2020.json.gz',
 'Product_jugais.com_September2020.json.gz',
 'Product_boogemes.com_September2020.json.gz',
 'MusicRecording_zaiaband.com_September2020.json.gz',
 'Product_thatawesomeshirt.com_September2020.json.gz',
 'Restaurant_nashvilledelivers.com_September2020.json.gz',
 'Recipe_completelyketo.com_September2020.json.gz',
 'Person_carepirates.com_September2020.json.gz',
 'MusicRecording_mariachiaguilasdemexico.com_September2020.jso

In [5]:
small_test = all_tables[all_tables['filename'].isin(small_listeTe)]
small_test.to_csv('../../src/data/schemafiltereddata/TrainTestTables/Small/STest_Files.csv')
small_test

,filename,Class,num_rows,num_columns,num_columns_selected,num_NAs,percentage_col_NA,percentage_table_NA,sum_NA_sel_col,Book_aggregaterating,...,Restaurant_openinghoursspecification,Restaurant_pricerange,Restaurant_servescuisine,Restaurant_telephone,TVEpisode_episodenumber,TVEpisode_name,TVEpisode_partofseries,Small,Medium,Large
106,Book_calamo.com_September2020.json.gz,Book,15746.0,10.0,7.0,6174,0.193891,0.056014,43218.0,0,...,0,0,0,0,0,0,0,0.0,0.0,1.0
201,Book_follasnovas.com_September2020.json.gz,Book,1167.0,7.0,5.0,181,0.152528,0.031020,905.0,0,...,0,0,0,0,0,0,0,0.0,0.0,1.0
472,Book_norfolk-tree-surgeon.co.uk_September2020.json.gz,Book,235.0,11.0,9.0,0,0.000000,0.000000,0.0,1,...,0,0,0,0,0,0,0,0.0,0.0,1.0
510,Book_pirtukakurdi.com_September2020.json.gz,Book,993.0,8.0,5.0,218,0.188318,0.043907,1090.0,0,...,0,0,0,0,0,0,0,0.0,0.0,1.0
572,Book_slkbali-lollypop.com_September2020.json.gz,Book,52.0,13.0,9.0,7,0.057692,0.014957,63.0,0,...,0,0,0,0,0,0,0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53789,Restaurant_laterrassedujardin.com_September2020.json.gz,Restaurant,22.0,13.0,9.0,11,0.363636,0.055556,99.0,0,...,1,1,1,1,0,0,0,1.0,0.0,0.0
53835,Restaurant_nashvilledelivers.com_September2020.json.gz,Restaurant,94.0,9.0,6.0,2,0.021277,0.003546,12.0,0,...,0,1,1,1,0,0,0,0.0,1.0,0.0
53935,Restaurant_tahoesbest.com_September2020.json.gz,Restaurant,32.0,10.0,7.0,5,0.093750,0.022321,35.0,0,...,0,1,1,1,0,0,0,1.0,0.0,0.0
54035,TVEpisode_erosnow.com_September2020.json.gz,TVEpisode,21.0,10.0,2.0,0,0.000000,0.000000,0.0,0,...,0,0,0,0,0,1,1,1.0,0.0,0.0
